# Welcome to MoSeq2-Notebook

### Run all of the MoSeq2 tools in a self-contained notebook.

***
<center><h1>MoSeq2 Introduction</h1></center>

***

<img src="https://drive.google.com/uc?export=view&id=1Cps4eniKXpoKwSjFGC4R7S1JSeG4bVJg">


MoSeq2 software toolkit for unsupervised characterization of animal behavior. Moseq takes depth recordings of single behaving animals as input, and outputs a rich labeling of postural dynamics in terms of reused motifs or 'syllables'. 

This notebook begins with compressed depth recordings (see 'Data Acquisiting Overview' below) and transforms this data through the steps of:
- **Extraction**: The animal is segmented from the background and its position and heading direction are aligned across frames.
- **Dimensionality reduction**: Raw video is de-noised and transformed to low-dimensional pose trajectories using principal component analysis (PCA).
- **Model training**: Pose trajectories are modeled using an autoregressive hidden Markov model (AR-HMM), producing a sequence of syllable labels.
- **Analysis**: Model output is reported through visualization and statistical analysis.

### Resources
Below are a list of publications and links to the individual github tool wikis for your convenience.
- Publications
    - [Mapping Sub-Second Structure in Mouse Behavior](http://datta.hms.harvard.edu/wp-content/uploads/2018/01/pub_23.pdf)
    - [The Striatum Organizes 3D Behavior via Moment-to-Moment Action Selection](http://datta.hms.harvard.edu/wp-content/uploads/2019/06/Markowitz.final_.pdf)
    - [Q&A: Understanding the composition of behavior](http://datta.hms.harvard.edu/wp-content/uploads/2019/06/Datta-QA.pdf)
- Wikis
    - [Extract](https://github.com/dattalab/moseq2-extract/wiki)
    - [PCA](https://github.com/dattalab/moseq2-pca/wiki)
    - [Model](https://github.com/dattalab/moseq2-model/wiki)
    - [Viz](https://github.com/dattalab/moseq2-viz/wiki)

## Data Acquisition Overview

Moseq2 takes animal depth recordings as input. We we have developed a [data acquisition pipeline](https://github.com/dattalab/moseq2-docs/wiki/Setup:-acquisition-software) for the Xbox Kinect depth camera. We suggest following our [data acquisiting tutorial](https://github.com/dattalab/moseq2-docs/wiki/Acquisition) for doing recordings. 

__It is recommended that MoSeq2 users collect over 10 hours of depth videos (nframes >= ~1 million frames) for the best analysis results.__

***

### Ensure you are running the python version located in your corresponding conda environment.

Remember: The anaconda environment must be activated prior to launching this jupyter notebook in order to use the specified python version.

For example, if your anaconda environment is called moseq2, then your output would look like: ```/Users/username/anaconda3/envs/moseq2/bin/python```

In [ ]:
%%bash
which python

***
<center><h1>Notebook Setup</h1></center>

***

Install the requirements for Moseq2 by following the [README file](http://localhost:8888/notebooks/MoSeq2_Step_0.ipynb) (if you have not done so already). Then input your desired data directory __absolute path__ in the cell below to get started.
<img src="https://drive.google.com/uc?export=view&id=1Rs-LYyYIHueyE3x60dKk_fTtNMkCzqSb">

### Session Folder Contents

After acquiring some data, an individual session folder should contain the following files:

```
├── session_1/ **
├   ├── depth.dat **        # depth data
├   ├── depth_ts.txt **     # timestamps
└─  └── metadata.json **    # metadata
```
***

- `depth.dat`: compressed file version of depth video (to be extracted)
- `depth_ts.txt`: timestamp file used for metadata analysis (can be used to check dropped fram e rate). Check the depth_ts.txt file to be sure that the dropped frame rate is less than 1%. If you look at the difference between neighboring timestamps, it should be ~30 (as in 30 milliseconds).
- `metadata.json`: JSON file containing recording session information.

__Keep each session recording folder separate in order to preserve the correct data parings.__

### Data file organization

You will be able to access your acquired data from any directory in your filesystem within this notebook.

To ensure that your Moseq2-Notebook runs smoothely, ensure your data directories follow the sample structure below:
- Organize your directories such that you have 1 parent directory that contains all of your individual session recording subdirectories (as shown below)

```
.
└── Data_Directory/
    ├── session_1/ **
    ├   ├── depth.dat        # depth data
    ├   ├── depth_ts.txt     # timestamps
    ├   └── metadata.json    # metadata
    ...
    ├── session_2/ **
    ├   ├── depth.dat
    ├   ├── depth_ts.txt
    └── └── metadata.json

```

## Notebook Progress File

In case that your notebook kernel is interrupted, crashes, etc. The MoSeq2-Notebook comes with a progress file that will store all of your variables within each session in case you need to restore them at a later time (without performing all the computations again).

The variables values will be written to the progress yaml file as you run each cell, such that when you return to your analysis session, you can run the following cell and load all your found variables. 

__The first cell of each section will be a convenience cell that will restore your notebook variables when run. You only need to run them if you have an uninitialized variable that has been previously computed in a previous kernel.__

### Find Your Recording Directories and Create The Progress File

Input the absolute path to your desired parent directory containing your recorded session subdirectories in the path variable in the cell below.

__Note: In order to correctly find your inputted directory, ensure to input the absolute path. The absolute path is the complete path to your file/folder in your current filesystem. To obtain it, you can run `pwd` in your bash terminal in your desired directory.__

The following cell will recursively search through your inputted directory and return a list of the found `.dat` files, and your base working directory path. Leaving the path empty will result in searching your current directory (where the notebook is located).

In [ ]:
from moseq2_extract.gui import get_found_sessions
from glob import glob
import ruamel.yaml as yaml
import os, sys

data_path = '/Users/aymanzeine/Desktop/Playground/test_sessions/'

base_dir, found_sessions = get_found_sessions(data_path)
progress_filepath = os.path.join(base_dir, 'test_progress.yaml')

print('Number of found sessions to analyze:', found_sessions)
print('Your base directory is:', base_dir)

if os.path.exists(progress_filepath):
    with open(progress_filepath, 'r') as f:
        progress_vars = yaml.safe_load(f)
    f.close()
    
    print('Progress file found, listing initialized variables...\n')
    
    for k,v in progress_vars.items():
        if v != 'TBD':
            print(f'{k}: {v}')
    
    restore = input('Would you like to restore the above listed notebook variables? [Y -> yes, else -> exit]')
    
    if restore != "Y":
        sys.exit()

    print('Updating notebook variables...')

    config_filepath = progress_vars['config_file']
    index_filepath = progress_vars['index_file']
    pca_dirname = progress_vars['pca_dirname']
    scores_filename = progress_vars['scores_filename']
    model_path = progress_vars['model_path']
    scores_file = progress_vars['scores_path']
    crowd_dir = progress_vars['crowd_dir']

else:
    print('Progress file not found, creating new one.')
    progress_vars = {'base_dir': base_dir, 'config_file': 'TBD', 'index_file': 'TBD', 'pca_dirname': 'TBD',
                    'scores_filename': 'TBD', 'scores_path': 'TBD', 'model_path': 'TBD', 'crowd_dir': 'TBD'}
    
    with open(progress_filepath, 'w') as f:
        yaml.safe_dump(progress_vars, f)
    f.close()

    print('\nProgress file created, listing initialized variables...')
    for k,v in progress_vars.items():
        if v != 'TBD':
            print(k, v)

### Generate Configuration Files

In [ ]:
import os
from moseq2_extract.gui import generate_config_command, update_progress

config_filepath = os.path.join(base_dir, 'config.yaml')

print(f'generating file in path: {config_filepath}')
generate_config_command(config_filepath)

update_progress(progress_filepath, 'config_file', config_filepath)

A configuration file has been created in your base directory (depicted below).

```
.
└── Data_Directory/
    ├── config.yaml **
    ├── session_1/ 
    ├   ├── depth.dat        
    ├   ├── depth_ts.txt     
    ├   └── metadata.json    
    ...
    ├── session_2/ 
    ├   ├── depth.dat
    ├   ├── depth_ts.txt
    └── └── metadata.json
```

### Download a Flip File

In order to ensure your extraction is smooth and invariant to the mouse's orientation, we recommend using a flip-classifier to aid keeping the mouse oriented throughout the extraction.
Note: MoSeq2-Notebook currently only supports flip correction for Adult male c57 mice. (You may skip this step if your mice are very different)

The following cell will also output the path where your flip classifier has been saved.

In [ ]:
from moseq2_extract.gui import download_flip_command

download_flip_command(base_dir, config_filepath)

***
<center><h1>Raw Data Extraction</h1></center>

***

You will use the MoSeq2-Extract module to convert your raw data files to human-readable/parseable formats such as mp4 videos, and YAML/HDF5 metadata files. These metadata files are used to then train your PCA model, while the mp4 file is primarily used to ensure that the session was extracted correctly with no defects or unwanted artifacts.

In the extraction step, begin by testing your detected ROIs with the default parameters. If your arena has been correctly detected, continue into the to the test extraction step.

The first two steps are meant to debug possible extraction errors you may encounter before performing an extraction on your full dataset.

This step is important because it will determine the integrity of your data going into the analysis steps. If the resulting extraction contains artifacts, or is too noisy, etc. the PCA and ARHMM modeling results will not be reliable.

<img src="https://drive.google.com/uc?export=view&id=1YBBpvLTq8xc6wzvQqrenGA9B6a01SH4L">

Once testing is done, you can then proceed to extract all the session files found by your notebook.

## Pre-Extraction Data Quality Testing

Before performing a full extraction on your recordings, ensure your Regions of Interest (ROIs) are properly found. This will bring more clarity as to what to expect after a complete extraction of your data.

The resulting calculated ROIs correspond to the detected arena floor, the background subtracted from the detected arena, and the first frame.

### ROI Test

This test ensures that your whole background area is properly captured without any artifacts that may interfere with the mouse video extraction.

### Configurable Parameter Descriptions
- `bg_roi_dilate`: the detected floor mask is dilated with a kernel whose (width, height) is specified by this parameter
- `bg_roi_depth_range`: min/max depth values in which the ROI detection algorithm will search for a flat surface.
- `bg_roi_gradient_threshold`: threshold value to decide which areas to include in the background roi.

***

### Possible ROI Pathologies

__Pathology__: Resulting background ROI contains holes, or walls are not crisp/well defined.

__Reasons__: 
- The recording environment may have been too bright, causing the infrared depth calculations to be skewed.
- The background depth range may be inaccurate to that of the real-life recording scenario. Always ensure that the inputted heights and depth ranges are accurate to that of the recorded session.

__Solutions__: 
- Assuming the data acquisition went okay, try adjusting the background dilation parameter to a larger kernel in the case of an incomplete background, and smaller in the case of ROIs with holes in the arena background.
- Similarly, check if the inputted depth range is accurate to that of the real-life recording camera height from the bucket floor.
- If the walls are not being captured correctly, or there are holes in the background, try __slightly__  decreasing the background gradient threshold (by increments of [50-100]) to try to capture more of the environment. 

***



The following cell will extract the first frame, ROI, and background ROI for your reference before continuing into the extraction process.

In [ ]:
import ruamel.yaml as yaml
from moseq2_extract.gui import find_roi_command

with open(config_filepath, 'r') as f:
    config_data = yaml.safe_load(f)
f.close()

# Relevant ROI parameters you may need to configure
config_data['bg_roi_dilate'] = (10, 10) # Size of the mask dilation (to include environment walls)
config_data['bg_roi_depth_range'] = (650, 750) # Range to search for floor of arena (in mm)
config_data['bg_roi_gradient_threshold'] = 3000 # Threshold at which ROI boundaries are placed on the input frame

with open(config_filepath, 'w') as f:
    yaml.safe_dump(config_data, f)
f.close()

rois_dir = find_roi_command(base_dir, config_filepath)

Once complete, you can expect the following directory structure:

```
.
└── Data_Directory/
    ├── config.yaml 
    ├── session_1/ 
    ├   ├── proc/ **
    ├   ├   ├── bground.png & bground.tiff **
    ├   ├   ├── first_frame.png & first_frame.tiff **
    ├   ├   └── roi.png & roi.tiff ** 
    ├   ├── depth.dat        
    ├   ├── depth_ts.txt     
    ├   └── metadata.json    
    ...
    └── session_2/ 
```

Display your calculated ROI images below:

In [ ]:
from IPython.display import display, Image
from glob import glob

images = glob(os.path.join(rois_dir, '*.png'))
ims = [Image(im) for im in images]
for im, ip in zip(ims,images):
    print(ip.split('/')[-1])
    display(im)

### Sample Test Extraction 

Now that the ROIs have been successfully detected, you can now test whether the mouse is segmented from the arena and oriented correctly during the extraction process.

The extracted version of the mouse will appear on the top left-hand corner of the generated video. If the mouse is consistently facing rightward, then the extraction test is successful and you can proceed to extract your dataset with the set parameters.

Otherwise, ensure that your environmental parameters are correct and an appropriate amount of spatial and temporal filtering has been applied.

### Configurable Parameter Descriptions
- `min_height`: The shortest possible height that the mouse can be in your recordings. Important factor to include in order to properly estimate the minimum depth value during video construction.
- `max_height`: The tallest possible height your mouse can be in your recordings. Important factor to include in order to properly estimate the maximum depth value during video construction.
- `spatial_filter_size`: Median spatial filter applied to the raw video pre-extraction in order to get crisp mp4 files. The larger the kernel, the more granular your video will become. Be aware not to set it too high as to not lose video clarity. (Must be ODD)
- `temporal_filter_size`: Median temporal filter applied to the raw video pre-extraction in order to handle any frame drops or time irregularities in the compressed data. Only use if your videos appear to be laggy or have a noticable amount of frames dropped. (Must be ODD)

***

### Possible Extraction Pathologies
__Pathology__: The extracted mouse image is too grainy causing the depth contour boundaries to not be well defined.

__Reasons__: 
- The inputted minimum and maximum mouse heights were inaccurate, causing the depth estimations to be incorrect.
- Spatial filter kernel size may be too small, underprocessing each frame.
- Temporal filter kernel size may be too large, overprocessing the mouse boundaries.

__Solutions__: 
- Ensure your mouse measurements are accurate to your recording environment.
- Try increasing the applied spatial kernel size. Increasing the kernel size may help make the mouse image crisp without losing definition in the body contours in the extracted image.
- Decreasing any applied temporal filtering is also recommended. However, note that there is a balance of temporal and spatial filtering to achieve a good extraction.

***

__Pathology__: Extracted mouse video is too jittery.


__Reasons__: 
- This is can be due to the lack of temporal filtering. 
- There may be too much noise in the image skewing the mouse ellipse fit, causing the centroid calculation to be inconsistent over frame ranges. 

__Solutions__: 
- Temporal filtering helps smoothen the transitions of the frames over time, simplifying the mouse ellipse-fit task, and therefore stabilizing the extracted video.
- Using an appropriate amount of spatial and temporal filtering will help remedy this issue.


***

__Pathology__: Extracted mouse orientation is not consistently correct. (Mouse is facing wrong direction)


__Reasons__: 
- Using too large of a tail filter size, or overfiltering the image to remove the mouse tail can also ambiguate the mouse ellipse fit, skewing orientation detection.
- Underfiltering the temporal domain can also lead to mouse orientation inaccuracies due to pixel jitter cause by the kinect on-line depth estimation. 
- Overfiltering the spatial domain can also cause inaccurate orientation flip detection due to the loss in detail of the image, namely around the edges of the mouse.


__Solutions__: 
- Ensure your flip classifier path is found in your `config.yaml` file in order for it to be applied.
- Temporal filtering helps keep the fitted ellipse consistent throughout the image processing, increasing orientation-flip precision.
- Once again, experiment to try to achieve the appropriate balance between temporal and spatial filtering.


***


In [ ]:
import ruamel.yaml as yaml
from moseq2_extract.gui import sample_extract_command

nframes = 200 # number of frames to extract from raw to preview

with open(config_filepath, 'r') as f:
    config_data = yaml.safe_load(f)
f.close()

# Extraction parameters you may need to configure
config_data['min_height'] = 10 # Min mouse height from floor (mm)
config_data['max_height'] = 100 # Max mouse height from floor (mm)
config_data['spatial_filter_size'] = [3] # Space prefilter kernel (median filter, must be odd)
config_data['temporal_filter_size'] = [0] # Time prefilter kernel (median filter, must be odd)

with open(config_filepath, 'w') as f:
    yaml.safe_dump(config_data, f)
f.close()

sample_ext_dir = sample_extract_command(base_dir, config_filepath, nframes)

After an extraction, you can expect the following directory structure:

```
.
├── config.yaml
├── session_1/
├   ├── sample_proc/ **
├   ├   ├── bground.png & bground.tiff **
├   ├   ├── first_frame.png & first_frame.tiff **
├   ├   ├── results_00.mp4 **
├   ├   ├── results_00.h5 **
├   ├   ├── results_00.yaml **
├   ├   └── roi.png & roi.tiff ** 
├   ├── depth.dat
├   ├── depth_ts.txt
├   └── metadata.json
└── session_2/
```

You can view your sample extraction below:

In [ ]:
from IPython.display import display, Video

vid = Video(os.path.join(sample_ext_dir, 'results_00.mp4'), embed=True)

display(vid)

If you are happy with your sample extraction, continue to extracting your full dataset. Otherwise, consider adjusting some of your ROI or extraction parameters.

## Extract Session(s)

The cell will prompt you to choose whether you would like to extract individual sessions, or all of them. Enter your selection, and then wait for the extraction to complete to preview them.

__WARNING: If you are extracting files of size X GB, where X much larger than the amount of available RAM GB, you may experience an interruption in the extraction due to a memory overflow problem.__

In [ ]:
from moseq2_extract.gui import extract_found_sessions

# depth files to recursively search for that have been partially extracted or not yet extracted 
ext = '.dat'

commands = extract_found_sessions(base_dir, config_filepath, ext)

This is what your directory structure should look like once the process is complete:

```
.
├── config.yaml
├── session_1/
├   ...
├   └── proc/ **
├   ├   ├── roi.tiff
├   ├   ...
├   ├   ├── results.yaml **
├   └   └── results.h5 **
└── session_2/
├   ...
├   └── proc/ **
├   ├   ├── roi.tiff
├   ├   ...
├   ├   ├── results.yaml **
└   └   └── results.h5 **
        
```

### Aggregate your results into one folder and generate an index file.

The following cell will search through your base directory for the `proc/` folders in each session, and copy them all in a single directory. 

Then it will generate the index file by searching for all the metadata found in the `results_00.h5`/`results_00.yaml` files, and consolidate all that information in one file, assigning each session to a `default` group.

#### Configurable Parameter Descriptions
- `recording_format`: the start_time, session_name, and subject_name parameters are variable names that are read from each `metadata.json` file. The names of the resulting files will have the inputted format.
- `aggregate_results_dirname`: directory name that contains all of your extracted data.

In [ ]:
from moseq2_extract.gui import aggregate_extract_results_command, update_progress

recording_format = '{start_time}_{session_name}_{subject_name}' # filename formats for the copied extracted data files
aggregate_results_dirname = 'aggregate_results/' # directory name to save all metadata+extracted videos to with above respective name format

index_filepath = aggregate_extract_results_command(base_dir, recording_format, aggregate_results_dirname)

update_progress(progress_filepath, 'index_file', index_filepath)

The aggregate results folder will be saved in your base directory,
resulting in the following directory (sample) structure where the base directory contains the notebook:

```
.
├── aggregate_results/ **
├   ├── session_1_results.h5 ** # session 1 metadata
├   ├── session_1_results.yaml **
├   ├── session_1_results.mp4 ** # session 1 extracted video
├   ├── session_2_results.h5 ** # session 2 metadata
├   ├── session_2_results.yaml **
├   └── session_2_results.mp4 ** # session 2 extracted video
├── config.yaml
├── moseq2-index.yaml ** # index file
├── session_1/
└── session_2/
```

__Notice your index file has been generated in your base directory.__

View all of your extracted videos below:

In [ ]:
from IPython.display import display, Video
from glob import glob
import os

extractions = glob(os.path.join(base_dir, aggregate_results_dirname, '*.mp4'))
vids = [Video(vid, embed=True) for vid in extractions]
for vid, ext in zip(vids, extractions):
    print(ext.split('/')[-1])
    display(vid)

***
<center><h1>Principal Component Analysis (PCA)</h1></center>

***

Once the data has been extracted, implement a Principal Component Analysis on your metadata (specifically h5 files) to compute the principal components of your mouse's body in order to subsequently classify its behavior in the ARHMM model.
<img src="https://drive.google.com/uc?export=view&id=1I1WcfEwzpfwIxNYStX7swLAIvjQEVApy">

## Training PCA

__A good example of what you should expect from your PCA Components and Scree plot are shown below:__

<center>Components</center> | <center>Scree Plot</center>
- | - 
<img src="https://drive.google.com/uc?export=view&id=1dX5Gpd3PKL4vfVviLeP0CqBrz9PW37Au" width=400 height=400> | <img src="https://drive.google.com/uc?export=view&id=12uqsBYuWCjpUQ6QrAjo35MnwYDzHqnge" width=400 height=400>

### Configurable Parameter Descriptions
- `gaussfilter_space`: Kernel size for performing a gaussian spatial filter on your processed mouse video before performing PCA. This helps identify crisper, more informative principal components.
- `gaussfilter_time`: Kernel size for performing a gaussian temporal filter
- `medfilter_space`: Same as gauss filter kernel but uses Median Filtering instead. (Typically use one or the other)
    - Both spatial filters are used for when the principal components do not appear to have crisp boundaries, or are all too similar to each other to be considered reliable components.
- `medfilter_time`: Same as gauss filter kernel but uses Median Filtering instead. (Typically use one or the other)
- `missing_data`: If you have missing/dropped frames in your videos, set this to true.
- `missing_data_iters`: Number of times to iterate over missing data during PCA to fill in missing gaps appropriately.
- `recon_pcs`: Number of principal components to reconstruct from missing data.

***

### Possible PCA Pathologies
__Pathology__: Cannot achieve a explained variance of over 90% from less than 15 Principal Components (PCs).

__Reasons__: 
- The crop size may be too large, resulting in large amounts of extraneous noise being included in the PCA model.
- The input frames may be too noisy; this could be noise stemming from overall image quality, over/under filtering, or non-consitstent mouse orientation or centroid.
- If the input frames are overly spatially smoothed, then the PCA will not be able to accurately separate each principal component.
- If the training set is overly temporally smoothed, then that could attribute to some overfitting principal components. This means that causing it to incorrectly explain a larger variance for a PC due to it being extentuated over more frames as a result of the temporal filter.
- Too few frames were inputted to derive proper PCs.

__Solutions__: 
- Check if the crop size is too large, if so, decrease it and re-extract your data.
- Try (incrementally) adjusting the spatial and temporal filtering kernel sizes in the PCA step. This will aid the PCA in constructing PCs that more accurately explain large enough pose trajectory portions to cover over 90% of the data's explained variance.
- Acquire and extract more data, then try with more data.

***

__Pathology__: Graphed PCs look overprocessed, or are not representative of realistic mouse body regions.


__Reasons__: 
- Lack of spatial filtering, and/or a large amount of pixel noise can attribute to this type of issue. Further, If the training set is too small for the large amount of features, then the PCA may try to overestimate some mouse body regions, causing inaccurate PC selection.
- Inaccurately inputted minimum and maximum mouse inputted heights could contribute to the over/underestimation of the depth contours’ explained variance, therefore segmenting the mouse’s body with very coarse boundaries.
- An excess of the combination of temporal and/or spatial filtering can also cause overprocessing of PCs due to the features not only have lost definition in the spatial domain, but also in the time domain.

__Solutions__: 
- Ensure that an appropriate amount of spatial and temporal filtering is applied.
- If there are  missing frames, apply and appropriate amount of temporal filtering.
- Ensure that if there are missing frames, a proper amount of PCs are being reconstructed (`recon_pcs` is set correctly).

***


### (Convenience) Restore Progress Variables

In [ ]:
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([18])")

***

In [ ]:
from moseq2_pca.gui import train_pca_command
from moseq2_extract.gui import update_progress
import ruamel.yaml as yaml

pca_filename = 'pca' # Name of your PCA model h5 file to be saved
pca_dirname = '_pca/' # Directory to save your computed PCA results

with open(config_filepath, 'r') as f:
    config_data = yaml.safe_load(f)
f.close()

# PCA parameters you may need to configure
config_data['gaussfilter_space'] = (1.5, 1) # Spatial filter for data (Gaussian)
config_data['medfilter_space'] = [0] # Median spatial filter
config_data['recon_pcs'] = 10 # Number of PCs to use for missing data reconstruction
config_data['missing_data'] = False # Use missing data PCA
config_data['missing_data_iters'] = 10 # Number of times to iterate over missing data during PCA


with open(config_filepath, 'w') as f:
    yaml.safe_dump(config_data, f)
f.close()

train_pca_command(base_dir, config_filepath, pca_dirname, pca_filename)

update_progress(progress_filepath, 'pca_dirname', pca_dirname)

Once complete, you can expect your relative directory structure to look something like this:
```
.
├── _pca/ **
├   ├── pca.h5 ** # pca model compressed file
├   ├── pca.yaml  ** # pca model YAML metadata file
├   ├── pca_components.png **
├   └── pca_scree.png **
├── aggregate_results/
├── config.yaml
├── moseq2-index.yaml
├── session_1/
└── session_2/

```

View your `computed PCs` and `scree plot` in the next cell.

In [ ]:
from IPython.display import display, Image
images = [os.path.join(base_dir, pca_dirname, 'pca_components.png'), 
          os.path.join(base_dir, pca_dirname, 'pca_scree.png')]
for im in images:
    display(Image(im))

## Computing Principal Component Scores

Apply your trained PCA model using your computed principal components to compute your PC Scores.

In [ ]:
from moseq2_pca.gui import apply_pca_command
from moseq2_extract.gui import update_progress

scores_filename = 'pca_scores' # name of the scores file to compute and save
index_filepath = os.path.join(base_dir, 'moseq2-index.yaml') # path to your auto-generated (possibly modified) index file

apply_pca_command(base_dir, index_filepath, config_filepath, pca_dirname, scores_filename)

update_progress(progress_filepath, 'scores_filename', scores_filename)

Once complete, you will have a pca_scores file saved in your pca directory. (Example shown below)
```
.
├── _pca/
├   ├── pca.h5
├   ├── pca.yaml
├   ├── pca_scores.h5  ** # scores file
├   ├── pca_components.png
├   └── pca_scree.png
├── aggregate_results/
├── config.yaml
├── moseq2-index.yaml
├── session_1/
└── session_2/

```

## (Optional) Computing Model-Free Syllable Changepoints

This is an optional step used to aid in determining model-free syllable lengths; which are general approximations of the duration of respective body language syllables. Computing Model-Free Changepoints can be useful for determining the prior variable for syllable duration, denoted as `kappa`, in the ARHMM modeling step.

__A good example of a Changepoints Distance plot is shown below__
<img src="https://drive.google.com/uc?export=view&id=1sMkSB34bGbOimumN6Gg1-zV2Hk98v2Zy" width=400 height=400>


Measure syllable block duration distances between detected syllables using your PCA model or computed PC scores below.

__Warning: These parameters have been pre-tuned to accomodate for C57 Mice, and those of the like. Therefore, we do not recommend changing the changepoint calculation parameters. However, if you decide to do so, it is at your own risk.__

### Configurable Parameter Descriptions
- `threshold`: Computed value used to determine the "peak"/transition point from one syllable to the other
- `dims`: Number of random projections to use in order to compare the computed principal components with, and determine a distribution for the block durations.

***

### Possible Model-Free Changepoints Pathologies
__Pathology__: Model-free syllable changepoint distances distribution is skewed incorrectly.

__Reasons__: 
- Considering that this a plot generated using pre-computed principal components, errors that appear here may be direcly correlated with a poorly fitted PCA.
- This can occur if PCA model was trained on input data with many missing frames that were not reconstructed using the correct PCs, iterated over too many or too few times, or did not use a proper mask configuration to properly reconstruct the missing frames.
- This can be attributed to an excess of temporal filtering, causing the computed principal components to be either misinterpreted or overgeneralized over long time scales, skewing the distance distribution.

__Solutions__: 
- Try retraining the PCA with less temporal filtering.
- Ensure your extracted data is correct. If the extraction version of the mouse is too noisy, then the PC trajectories cannot be accurately applied to the data.  
- Get more data and try again.

***

In [ ]:
from moseq2_pca.gui import compute_changepoints_command
import ruamel.yaml as yaml
changepoints_filename = 'changepoints' # name of the changepoints images to generate
with open(config_filepath, 'r') as f:
    config_data = yaml.safe_load(f)
f.close()

# Changepoint computation parameters you may want to configure
config_data['threshold'] = 0.5 # Peak threshold to use for changepoints
config_data['dims'] = 300 # Number of random projections to use

with open(config_filepath, 'w') as f:
    yaml.safe_dump(config_data, f)
f.close()

compute_changepoints_command(base_dir, config_filepath, pca_dirname, changepoints_filename)

The changepoints plot will be generated and saved in the pca directory (example below).

```
.
├── _pca/ 
├   ├── pca.h5
├   ├── pca_scores.h5
├   ...
├   └── changepoints_dist.png **
├── aggregate_results/ 
├── config.yaml
├── moseq2-index.yaml
├── session_1/
└── session_2/
```

View your changepoints distance plot:

In [ ]:
from IPython.display import display, Image

display(Image(os.path.join(base_dir, pca_dirname, changepoints_filename+'_dist.png')))

***
<center><h1>ARHMM Modeling</h1></center>

***

In order to train your ARHMM (Auto-Regressive Hidden Markov Model), you will use your computed PC scores as your input data, and specify whether you are modeling a single experimental group for observational research, or modeling multiple different groups (e.g. control vs. experimental groups) for comparative analysis.

<img src="https://drive.google.com/uc?export=view&id=1V2n5Jg61Pr86m0groyTX_qJ40bSm5LG7">

## (Optional) Specify Groups

### What are groups?

MoSeq using groups in the `moseq2-index.yaml` file to indicate whether your collected sessions are representing a single experimental group, or many different groups that you would like to compare while modeling and visuslizing

The index file requires that all your sessions have a metadata.json file in order to successfully assign each recorded subject or session to a group.

By default, all the session recordings have the same group title: `'default'`. If you do not have 2 sessions that are different enough to separate to different groups for later comparison, you can skip this step.

Otherwise, there are 3 ways you are able to specify your groups:
1. Specify group by SessionName
2. Specify group by SubjectName
3. Manually edit index file

Once a cell is run, it will display your current indexing structure.

### (Convenience) Restore Progress Variables

In [ ]:
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([18])")

### Check Your Index File
#### View Indexed Sessions
Use this cell to view your sessions' information regarding their SessionNames, SubjectNames, and Groups.

In [ ]:
from moseq2_viz.gui import get_groups_command

index_filepath = os.path.join(base_dir, 'moseq2-index.yaml')

get_groups_command(index_filepath)

#### 1 - Specify Group by Session Name

In [ ]:
from moseq2_viz.gui import add_group_by_session

value = 'ayman_first_tethered_recording' # value of the corresponding key
group = 'group2' # designated group name
exact = True # Must be exact key-value match
lowercase = False # change to lowercase
negative = False # select opposite selection than key-value pair given

add_group_by_session(index_filepath, value, group, exact, lowercase, negative)

#### 2 - Specify Group by Subject Name

In [ ]:
from moseq2_viz.gui import add_group_by_subject

value = 'blackStockOFA80GritSanded_012517' # value of the corresponding key
group = 'group2' # designated group name
exact = False # Must be exact key-value match
lowercase = False # change to lowercase
negative = False # select opposite selection than key-value pair given

add_group_by_subject(index_filepath, value, group, exact, lowercase, negative)

#### 3 - Manually Edit Index File

Simply navigate to your `moseq2-index.yaml` file in your designated directory, and editing the group key-value pair to your specified name values.

***

## Train ARHMM

### Configurable Parameter Descriptions
- `hold_out`: Boolean for whether to hold out data during the training process.
- `hold_out_seed`: Integer used to reproduce the same hold out set for repeated testing.
- `nfolds`: Number of data folds to hold out during training. (If used, nfolds <= nsessions)
- `npcs`: Number of selected principal components, chosen in order as shown in the PC Components plot. If too few or too many PCs are selected, the ARHMM predictions will become unreliable.
- `num_iter`: Number of time the model will iterate over your dataset, we recommend at least 100 starting out. This is modeling regularization parameter to ensure that your model is fitting appropriately to its given dataset.
- `max_states`: Maximum number of states the ARHMM that the ARHMM can end up with at the end of training. This is modeling regularization parameter that indicates the complexity of the transitions that may be happening in your dataset. Therefore, if there are too few the model may not learn the actual behavior, and if there are too many, then the model will overfit to the dataset.
- `separate-trans`: Boolean for whether to separate the modeling process for different groups. (Must set to true if number of unique groups > 1)
- `kappa`: Prior probability distribution variable used to indicate average syllable length. Setting kappa to the number of frames is a good starting point to determining the proper expressed syllable durations. If kappa is too low, syllables will shorter in duration, and vice versa.
- `checkpoint_freq`: Value indicating when to save model checkpoints per number of iterations passed. (If -1, do not checkpoint)

In [ ]:
from moseq2_model.gui import learn_model_command
from moseq2_extract.gui import update_progress
import os

pca_dir = os.path.join(base_dir, pca_dirname)
scores_file = os.path.join(pca_dir, scores_filename+'.h5') # path to input PC scores file to model
model_path = os.path.join(base_dir, 'model.p') # path to save trained model

# Advanced modeling parameters
hold_out = False # boolean to hold out data during the training process
hold_out_seed = 42 # integer to standardize the held out folds during training
nfolds = 2 # number of folds to hold out during training (if hold_out==True)
npcs = 10  # number of PCs being used

num_iter = 100 # number of iterations to train model
max_states = 100 # number of maximum states the ARHMM can end up with
kappa = None # syllable length prior None for default=nframes
robust = False # use robust-ARHMM with t-distribution

separate_trans = True # separate group transition graphs; set to True if ngroups > 1

checkpoint_freq = -1 # model saving freqency (in interations)

learn_model_command(scores_file, model_path, config_filepath, index_filepath, hold_out, nfolds,
                    num_iter, max_states, npcs, kappa,
                    separate_trans, robust, checkpoint_freq)

update_progress(progress_filepath, 'scores_path', scores_file)
update_progress(progress_filepath, 'model_path', model_path)

Once training is complete, your model will be saved in your base directory (shown below). 
```
.
├── _pca/ 
├── aggregate_results/ 
├── config.yaml
├── model.p **
├── moseq2-index.yaml/
├── session_1/
└── session_2/
```

Now use the moseq2-viz module to produce crowd videos and a number of statistical analysis plots.

***
<center><h1>Visualize Analysis Results</h1></center>

***

Now that you have a trained ARHMM, you can use it generate informative graphs and videos regarding the behavior syllables found, their usage frequency, and transition probabilities.

The graph below shows the 4 operations that the MoSeq2-Viz module currently affords. They can also be computed in any order at this point in the notebook.

<img src="https://drive.google.com/uc?export=view&id=1K_PnJ6psGxmXkUScIpfBStzXbCTO6MB2">

## Make Crowd Videos

This tool allows you to create videos containing many overlayed clips of the mouse performing the same specified syllable at the moment a red dot appears on their body. The videos are sorted by most frequently expressed syllable to least.

***

### Possible Crowd Movie Pathologies

__Pathology__: Generated crowd movies look all too similar.

__Reasons__: 
- The input data size (number of frames) is too small, causing the ARHMM to overfit each state to a larger set of PCs.
- The selected PCs may have been overfiltered, causing them to be fitted to a larger set of differently labelled syllables.
- The ARHMM model may be too simple, meaning that either the selected PCs do not explain a large enough amount of variance in the set, or that the selected maximum amount of states is not enough to capture the full variance in the mouse’s behaviors.


__Solutions__: 
- Ensure your PCs cover over 90% the data's explained variance, and that they are all included in the ARHMM training data.
- Try increasing the number of `max_states` that the ARHMM has to increase the problem complexity, increasing variance in labels.
- Try increasing the number of PCs being used in the ARHMM training.

***

__Pathology__: Generated crowd movies do not contain mice unanimously exhibiting the same syllable, or have large varying time-scales.


__Reasons__: 
- The ARHMM model is underfitted; this could be due to too few model training iterations, or the max number of states could be too large, hence incorrectly predicting the mouse’s behaviors because the syllable likelihoods have not yet converged at the end of training.
- The selected syllable duration prior probability distribution, `kappa`, denotes the syllable time series duration probability distribution. If it is too large, then the resulting syllables will span over large time scales, and vice versa.
- The input frames may be too noisy, or the PC’s selected may be too complex/unrealistic to properly represent a consistent behavior syllable.
- Too many PCs may have been selected, causing the dimensionality complexity to increase, resulting in overly explained syllables.
- Too much additive noise was augmented during the training process, possibly also during the PCA training, overcomplicating the PC-to-time-series syllable mapping problem.
- Improper temporal smoothing will lead to unreliable calculations of syllable prior distributions.


__Solutions__: 
- Increase the number of model training iterations in order ensure that the syllable likelihoods are converging. 
- Try decreasing the complexity of your ARHMM. If the `max_states` is set too high, you may end up with many 'starved states' that will skew the posterior probability distributions for some syllables.
- Try decreasing the amount of additive noise (if added), or add spatial filtering to your PC or frame data.
- Acquire more data and try again.

***


__Note: ensure all of the paths listed the `moseq2-index.yaml` file are correct in order for the following cells to run successfully.__

If the command stalls, restart the kernel, reload your variables and run it again.

To create the crowd videos, run the following command:

### (Convenience) Restore Notebook Variables

In [ ]:
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([18])")

In [ ]:
import os
from moseq2_viz.gui import make_crowd_movies_command
from moseq2_extract.gui import update_progress

model_path = os.path.join(base_dir, 'model.p') # path to save trained model
index_filepath = os.path.join(base_dir, 'moseq2-index.yaml')
crowd_dir = os.path.join(base_dir, 'crowd_movies/') # output directory to save all movies in

max_syllables, max_examples = 20, 20 # maximum number of syllables, and examples of each syllable in a video respectively

make_crowd_movies_command(index_filepath, model_path, config_filepath, crowd_dir, max_syllables, max_examples)

update_progress(progress_filepath, 'crowd_dir', crowd_dir)

Once completed, you can find your crowd movies along with a metadata YAML file in your corresponding crowd directory. The metadata `info.yaml` file will contain model information pertaining to how these crowd videos were produced.
```
.
├── _pca/ 
├── aggregate_results/ 
├── config.yaml
├── crowd_movies/ **
├   ├── info.yaml **
├   ├── syllable_sorted_44 (usage).mp4 **
├   ...
├   └── syllable_sorted_12 (usage).mp4 **
├── model.p 
├── moseq2-index.yaml
├── session_1/
└── session_2/
```

View your generated crowd movies below:

In [ ]:
from IPython.display import display, Video
from glob import glob

videos = glob(os.path.join(crowd_dir, '*.mp4'))
vids = [Video(vid, embed=True) for vid in videos]
for vid, vp in zip(vids, videos):
    print(vp.split('/')[-1])
    display(vid)

## Compute Usage Plots

Use this command to compute the model-detected syllables usages sorted in descending order of usage. 

__Note for plotting multiple groups: remember to include all group names in the tuple to graph them. Additionally, the model must be already trained on the specified groups. (When training the model, it will first output the uuids that have been added with their specified group name for your reference).__

In [ ]:
from moseq2_viz.gui import plot_usages_command

model_path = os.path.join(base_dir, 'model.p')
plot_path = os.path.join(base_dir, 'plots/')
sort = True
count = 'usage'
max_used_syllable = max_syllables - 1 
groups = ('group1', 'group2') #empty tuple for single group usage plot
output_file = os.path.join(plot_path, 'usages')

if not os.path.exists(plot_path):
    os.makedirs(plot_path)

plot_usages_command(index_filepath, model_path, sort, count, max_syllables, groups, output_file)

## Compute Scalar Summary and Tracking Plots

Use the following command to compute some scalar summary information about your modeled groups, such as average velocity, height, etc.
This command will also generate a tracking summary plot; depicting the path traveled by the mouse in your recordings.

In [ ]:
from moseq2_viz.gui import plot_scalar_summary_command

output_file = os.path.join(plot_path, 'scalars') # prefix name of the saved scalar position and summary graphs

plot_scalar_summary_command(index_filepath, output_file)

Graph your output:

In [ ]:
from IPython.display import display, Image
from glob import glob

images = glob(os.path.join(plot_path, 'scalars_*.png'))
ims = [Image(im) for im in images]
for im in ims:
    display(im)

## Compute Syllable Transition Graph

Use the following command to generate a syllable transition graph. The graph will be comprised of nodes labelled by syllable, and edges depicting a probable transition, with edge thickness depicting the weight of the transition edge.

***

### Possible Syllable Transition Graph Pathologies

__Pathology__: Too little syllables are being generated in the syllable transition graph.

__Reasons__: 
- If the ARHMM is trained on too few states or principal components, then the syllable mapping problem becomes oversimplified, causing less syllables to be recognized.
- If the syllables probability distribution is too sporatic, the model will have a high propensity to get stuck in local altimas during training, leading to underfitting.
- The input data (frames and/or PCA) may have been excessively spatially filtered, losing the spatial syllable complexity, leading to the model overgeneralizing the mouse poses and detecting less syllables. Conversely, it could have a large amount of dropped frames, leading to the same problem.
- The weights can be influenced by the `kappa` parameter, denoting how long a mouse will be exhibiting a given syllable. Inaccurate probability priors inputted during training can have catastrophic repercussions on model reliability.

__Solutions__: 
- A good way to combat this issue is to use a Robust AR-HMM, which normalizes the syllable probability distributions to a t-distribution.
- Ensure an appropriate amount of temporal filtering is applied to your extracted videos. If too much smoothing is applied then the transitions will become less apparent, skewing model predictions.
- Try using different values of `kappa` by increasing it or decreasing it by factors of 10. (Note: we recommend using a minimum `kappa` value equal to the number of frames).
- Try increasing the `max_states` variable to add more complexity to the model training problem.
- Adjusting any spatial or temporal filtering applied in the Extract and/or PCA steps (depending on which appears to be more noisy).

***

__Pathology__: Too many syllables being generated in the syllable transition graph, all with skewed weights.

__Reasons__: 
- The syllable transition probabilities may not have converged yet.
- The model syllable-transition probability distribution is too sporatic; this can be caused by an underfitted model, too many emptied hidden states, or noisy input data (PCs and/or frames).
- The input frames may either have too many dropped frames, or has been excessively temporally filtered, perturbing all the calculated syllable transition probability distributions.
- The default `alpha` parameter, denoting syllable transition prior probability distributions, may not be representative of the real life data being sampled.

__Solutions__: 
- Using a `robust` t-distribution model will aid in normalizing state posterior probability distributions.
- Increase the number of training iterations.
- Increasing `nfolds` and holding out data during training (1 fold per recording in the dataset). This will ensure that the model will not overfit by estimating validation log-likelihoods to compare to during training. It is recommended to hold out 1 fold per recording such that the validation set is properly representative of your training data.

***

For multiple groups, there will be a transition graph for each group, as well as a unified graph with different colors to identify the groups. __Note: remember to include all group names in the tuple to graph them.__

In [ ]:
from moseq2_viz.gui import plot_transition_graph_command

max_syllable = 20 # Maximum number of nodes in the transition graph
groups = ('group1', 'group2') # Group(s) to graph, default graph if empty tuple
output_filename = os.path.join(plot_path, 'transition') # name of the png file to be saved

plot_transition_graph_command(index_filepath, model_path, config_filepath, max_syllable, groups, output_filename)

***
<center><h1>Notebook End</h1></center>

***